# CSE 144 Group 3
## Music Recommendation System (MRS)

In this notebook, we write the predictive model for our music recommendation system. Our work leverages modern tools including recurrent neural networks (RNN) and BERT sentence transformers...

<br>

Our work leverages this RNN model:

https://github.com/taylorhawks/RNN-music-recommender/blob/master/cloud/model.ipynb


In [35]:
# import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
%config InlineBackend.figure_format="retina"
import numpy as np
import random
import torch
import os
# from torch import nn, optim
# import math
# from IPython import display
# import torchvision.datasets as datasets
# import torchvision.transforms as transforms
# from torch.utils.data import TensorDataset
# import torch.nn.functional as F
# from sklearn.preprocessing import MinMaxScaler
# import pdb
import plotly.graph_objects as go
import numpy as np

from skimage.util.shape import view_as_windows as viewW
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.decomposition import PCA

# import tensorflow as tf

# import keras.backend as K
from keras.models import Sequential, load_model
# from keras.optimizers import RMSprop
from keras.layers import Dense, SimpleRNN, Input
from keras.losses import *


### Load the data

In [36]:
song_features_data = pd.read_csv('misc/processed_music_info_extended.csv')
user_listening_data = pd.read_csv('misc/processed_user_listening_hist.csv')

# from google.colab import drive
# drive.mount('/content/drive')
# import pandas as pd
# song_features_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/music_info.csv')
# user_listening_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/user_listening_hist.csv')

### Set Random Seed

In [37]:
torch.manual_seed(24)

### Read and Display Data

In [38]:
print('# of rows of Song Data: ' + str(len(song_features_data)))
print('# of unique songs: ' + str(len(song_features_data['track_id'].unique())))
song_features_data.head()

# of rows of Song Data: 50683
# of unique songs: 50683


,track_id,name,artist,spotify_id,tags,year,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,TRIOREW128F424EAF0,Mr. Brightside,The Killers,09ZQ5TmUG8TSL56n0knqrj,"rock, alternative, indie, alternative_rock, in...",2004,222200,0.355,0.918,1,-4.360,1,0.0746,0.001190,0.000000,0.0971,0.240,148.114,4
1,TRRIVDJ128F429B0E8,Wonderwall,Oasis,06UfBBDISthj1ZJAtX4xjj,"rock, alternative, indie, pop, alternative_roc...",2006,258613,0.409,0.892,2,-4.373,1,0.0336,0.000807,0.000000,0.2070,0.651,174.426,4
2,TROUVHL128F426C441,Come as You Are,Nirvana,0keNu0t0tqsWtExGM3nT1D,"rock, alternative, alternative_rock, 90s, grunge",1991,218920,0.508,0.826,4,-5.783,0,0.0400,0.000175,0.000459,0.0878,0.543,120.012,4
3,TRUEIND128F93038C4,Take Me Out,Franz Ferdinand,0ancVQ9wEcHVd0RrGICTE4,"rock, alternative, indie, alternative_rock, in...",2004,237026,0.279,0.664,9,-8.851,1,0.0371,0.000389,0.000655,0.1330,0.490,104.560,4
4,TRLNZBD128F935E4D8,Creep,Radiohead,01QoK9DA7VTeTSE3MNzp4I,"rock, alternative, indie, alternative_rock, in...",2008,238640,0.515,0.430,7,-9.935,1,0.0369,0.010200,0.000141,0.1290,0.104,91.841,4


In [39]:
print('# of rows of User Listening Data: ' + str(len(user_listening_data)))
print('# of unique users: ' + str(len(user_listening_data['user_id'].unique())))
user_listening_data.head()

# of rows of User Listening Data: 806745
# of unique users: 25343


,track_id,user_id,playcount
0,TRLATHU128F92FC275,5a905f000fc1ff3df7ca807d57edb608863db05d,11
1,TRMKFPN128F42858C3,5a905f000fc1ff3df7ca807d57edb608863db05d,2
2,TRGAOLV128E0789D40,5a905f000fc1ff3df7ca807d57edb608863db05d,2
3,TREAQSX128E07818CA,5a905f000fc1ff3df7ca807d57edb608863db05d,2
4,TRUMDRI128F424FEFC,5a905f000fc1ff3df7ca807d57edb608863db05d,3


### Data Preprocessing


In [40]:
# Drop unnecessary columns
song_features_data = song_features_data.drop(columns=['year', 'time_signature', 'key'])

In [41]:
# Convert song duration from milliseconds to minutes
song_features_data["duration_mins"] = song_features_data["duration_ms"] / 60000
song_features_data.drop("duration_ms", axis=1, inplace=True)


song_features_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50683 entries, 0 to 50682
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   track_id          50683 non-null  object 
 1   name              50683 non-null  object 
 2   artist            50683 non-null  object 
 3   spotify_id        50683 non-null  object 
 4   tags              49556 non-null  object 
 5   danceability      50683 non-null  float64
 6   energy            50683 non-null  float64
 7   loudness          50683 non-null  float64
 8   mode              50683 non-null  int64  
 9   speechiness       50683 non-null  float64
 10  acousticness      50683 non-null  float64
 11  instrumentalness  50683 non-null  float64
 12  liveness          50683 non-null  float64
 13  valence           50683 non-null  float64
 14  tempo             50683 non-null  float64
 15  duration_mins     50683 non-null  float64
dtypes: float64(10), int64(1), object(5)
memo

In [42]:
data = pd.merge(song_features_data, user_listening_data, on='track_id')
data.head()

,track_id,name,artist,spotify_id,tags,danceability,energy,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_mins,user_id,playcount
0,TRIOREW128F424EAF0,Mr. Brightside,The Killers,09ZQ5TmUG8TSL56n0knqrj,"rock, alternative, indie, alternative_rock, in...",0.355,0.918,-4.360,1,0.0746,0.001190,0.0,0.0971,0.240,148.114,3.703333,fe31db6d197a667d265ff5a35d80d60f3660f729,2
1,TRRIVDJ128F429B0E8,Wonderwall,Oasis,06UfBBDISthj1ZJAtX4xjj,"rock, alternative, indie, pop, alternative_roc...",0.409,0.892,-4.373,1,0.0336,0.000807,0.0,0.2070,0.651,174.426,4.310217,67874d1a189c83326c529e554be6f7acf55effae,12
2,TRRIVDJ128F429B0E8,Wonderwall,Oasis,06UfBBDISthj1ZJAtX4xjj,"rock, alternative, indie, pop, alternative_roc...",0.409,0.892,-4.373,1,0.0336,0.000807,0.0,0.2070,0.651,174.426,4.310217,e3ee8846c9a5a0916700a9e7abfc1c5b2fcb8e36,5
3,TRRIVDJ128F429B0E8,Wonderwall,Oasis,06UfBBDISthj1ZJAtX4xjj,"rock, alternative, indie, pop, alternative_roc...",0.409,0.892,-4.373,1,0.0336,0.000807,0.0,0.2070,0.651,174.426,4.310217,cbb6b8dccf0af0d221dfd4684072c04bb0346f30,2
4,TRRIVDJ128F429B0E8,Wonderwall,Oasis,06UfBBDISthj1ZJAtX4xjj,"rock, alternative, indie, pop, alternative_roc...",0.409,0.892,-4.373,1,0.0336,0.000807,0.0,0.2070,0.651,174.426,4.310217,2cdf67cd70a64964cb914835af0043fcc28a8f48,12


### Obtain total number of listens per song

In [43]:
play_counts = data.groupby('name')['playcount'].sum().reset_index()
play_counts

,name,playcount
0,#1 Zero,13
1,#16,110
2,#17,7
3,#24,5
4,$20 for Boban,43
...,...,...
23579,慟哭と去りぬ,134
23580,我、闇とて･･･,7
23581,朔-saku-,51
23582,蜷局,368


### Create playlists for input to RNN

In [44]:
data = data.sort_values(['user_id'])
data

,track_id,name,artist,spotify_id,tags,danceability,energy,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_mins,user_id,playcount
306346,TRTVXIH128F426625A,Come Round Soon,Sara Bareilles,0jkVXytWSisMUtrBEej9mi,"pop, female_vocalists, singer_songwriter, soul...",0.338,0.819,-4.495,0,0.0776,0.077700,0.000000,0.1590,0.545,74.751,3.552000,0000f88f8d76a238c251450913b0d070e4a77d19,2
417455,TRWUFEW128F14782F3,Forever My Friend,Ray LaMontagne,0Ev7atdl0qS2n39OO7051O,"folk, singer_songwriter, soul, blues, acoustic...",0.493,0.524,-13.553,1,0.0423,0.334000,0.014100,0.3570,0.379,176.233,5.788883,0000f88f8d76a238c251450913b0d070e4a77d19,2
32466,TRNXEPE128F9339E47,My Name Is Jonas,Weezer,0YU04WSkTVomRgeDOWlEzX,"rock, alternative, indie, alternative_rock, in...",0.261,0.947,-3.031,1,0.0488,0.000197,0.003320,0.3100,0.550,185.942,3.435333,0000f88f8d76a238c251450913b0d070e4a77d19,2
698954,TRMKCCV128F92EB22E,Light On,David Cook,1BnoZbPDh9dbYqabvM6qZg,"rock, alternative_rock, male_vocalists",0.448,0.830,-4.156,0,0.0332,0.067300,0.000000,0.1130,0.362,131.991,3.816883,0000f88f8d76a238c251450913b0d070e4a77d19,3
227171,TRJGJTH128F4291A81,"Oh My God, Whatever, Etc.",Ryan Adams,0sUzPqm1gdsabzX5htMvf7,"rock, indie, folk, singer_songwriter, acoustic...",0.572,0.395,-10.630,1,0.0304,0.700000,0.000250,0.1260,0.483,79.552,2.532667,0000f88f8d76a238c251450913b0d070e4a77d19,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
802077,TRSEFCM128F429354D,Set It Up,Xavier Rudd,1sF9FiOQivhgedQnS1j3fK,"acoustic, 00s",0.469,0.385,-11.300,0,0.0270,0.503000,0.001390,0.1150,0.116,130.767,4.141550,fffbab4b8416fc41d05fcbdcf0e6735c4f37cb39,2
417463,TRWUFEW128F14782F3,Forever My Friend,Ray LaMontagne,0Ev7atdl0qS2n39OO7051O,"folk, singer_songwriter, soul, blues, acoustic...",0.493,0.524,-13.553,1,0.0423,0.334000,0.014100,0.3570,0.379,176.233,5.788883,fffbab4b8416fc41d05fcbdcf0e6735c4f37cb39,8
649627,TRDVGIH128F429353C,Come Let Go,Xavier Rudd,258CEuV9zzGk2PraoCH2yx,"reggae, male_vocalists",0.547,0.546,-8.634,1,0.0470,0.114000,0.037200,0.3810,0.280,140.477,6.870433,fffbab4b8416fc41d05fcbdcf0e6735c4f37cb39,28
553208,TROXFVJ128F1465265,Bottom Of the Barrel,Amos Lee,1VWGfrhpY8IiNmqMHavRXS,"folk, soul, acoustic, guitar",0.609,0.346,-12.703,1,0.1460,0.761000,0.000000,0.1100,0.550,178.137,2.006433,fffbab4b8416fc41d05fcbdcf0e6735c4f37cb39,4


In [45]:
# Changed name to track_id
playlists = data.groupby('user_id')['track_id'].apply(lambda x: list(x.head(20)))
playlist_dict = playlists.to_dict()
print(playlists)

user_id
0000f88f8d76a238c251450913b0d070e4a77d19    [TRTVXIH128F426625A, TRWUFEW128F14782F3, TRNXE...
0005eb11fd1dad47e6e6719a4db30340073a9e38    [TRGOJNK128F92F2A03, TRQPSHM128F92F29ED, TRTUW...
000d80cd9b58a8f77b33aa613dcfc5cbf1daf5e8    [TRDYYKS128F4275626, TRBHLYP12903D0D107, TRABF...
000e9296161b73a1821aaed3d7f50d95e8665bf6    [TROPEIV128F428F5A8, TRIAZQY128F934D58D, TRMKA...
00100482b3f3074549c751e718c57ed211b35991    [TRSNCIW128F14557BC, TRJKPFL12903CCE490, TRWJN...
                                                                  ...                        
fff7352d8ca192c451ce4fa00d18e33e261ecad3    [TRDRVJA128F4267831, TRCKWGF12903CD2DCD, TRXUW...
fff759a45a3a68de552740e8285a97d5f65d4e58    [TRDJZFF128F92D2627, TRULONW128F9302209, TRBNY...
fff9bd021bf6e07936883b9bb045207fcf372a2c    [TROHXCJ128F935A6AC, TRUMJNK12903CF465A, TRXYM...
fffb0b218640d86e5cb99d41cd3ecad977142da5    [TRZGGHL12903CDBF1F, TRCAUIX128F4277AD0, TRYIK...
fffbab4b8416fc41d05fcbdcf0e6735c4f37cb39    [TRGPCUN

In [46]:
# Changed track_id to name
data_dict = data.drop(['artist', 'tags', 'playcount'], axis=1)
# Changed name to track_id
data_dict = data_dict.set_index(['user_id', 'track_id']).to_dict('index')

In [47]:
songs_done = 0
updated_playlist_dict = {}
for user_id, songs in playlist_dict.items():
    updated_songs = []
    for song in songs:
        key = (user_id, song)
        if key in data_dict:
            the_features = list(data_dict[key].values())
            updated_songs.append([song] + the_features)
            songs_done += 1
            if songs_done % 10000 == 0:
                print(songs_done)
    updated_playlist_dict[user_id] = updated_songs

playlist_dict = updated_playlist_dict

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000


In [48]:
arr = []
for user_id, playlist in playlist_dict.items():
    arr2 = []
    for song in playlist:
        arr2.append(np.concatenate((song[0:6], song[7:12])))
    arr.append(arr2)

arr_np = np.array(arr)

In [49]:
playlists = pd.DataFrame.from_dict(playlist_dict, orient='index')
playlists.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0000f88f8d76a238c251450913b0d070e4a77d19,"[TRTVXIH128F426625A, Come Round Soon, 0jkVXytW...","[TRWUFEW128F14782F3, Forever My Friend, 0Ev7at...","[TRNXEPE128F9339E47, My Name Is Jonas, 0YU04WS...","[TRMKCCV128F92EB22E, Light On, 1BnoZbPDh9dbYqa...","[TRJGJTH128F4291A81, Oh My God, Whatever, Etc....","[TRFQYFT128F14840BC, Nobody Girl, 2YqjAMK5eeSk...","[TRKSXHR128F1455E4D, Dear Chicago, 2J8P81JjKem...","[TRGJTIY128F4296A0E, All You Need Is Love, 0BW...","[TRZLJOC128F14840BE, Enemy Fire, 13gnIRFWtBQN1...","[TRSMEUG128F14856D2, Within You, 0VRC5T7fDBY1S...","[TRZYESA128F148D67F, Please Do Not Let Me Go, ...","[TRRUNEV128F148D719, Burning Photographs, 2Mco...","[TRDKRLP128F4291A80, Halloweenhead, 04N3X9vfSz...","[TRFTUIW128E0784B9F, Bubble Toes, 1CFwwYZ58s34...","[TRKGCIA128F92C315D, Joe's Head, 0A2BgEzGWU9HB...","[TRFUCYR128F92DC67F, California Waiting, 0txCP...","[TROZZNY128F14782F7, All the Wild Horses, 0FFm...","[TRTWOCA128F14840B8, La Cienega Just Smiled, 0...","[TRQSEMJ128F4294F24, Pearls On A String, 02WVv...","[TRUNKTP12903CD1EFB, Blue Sky, 08SPbOlgCODbnWE..."
0005eb11fd1dad47e6e6719a4db30340073a9e38,"[TRGOJNK128F92F2A03, The Technicolor Phase, 27...","[TRQPSHM128F92F29ED, The Airway, 3Cy5wM1kAWdQ3...","[TRTUWMO128F92F2A09, Dear Vienna, 2LBdBoz94BqE...","[TRRNWAK128F92F29FB, Super Honeymoon, 0aMWS9ld...","[TRYEGSH12903CD2DCE, Overboard, 0cfsbkanGUO3yz...","[TRCKWGF12903CD2DCD, Never Let You Go, 7mP4fGw...","[TRNFVQI128F931BAEA, The Saltwater Room, 1eX8F...","[TRTKLFX12903CD2DC2, First Dance, 0OQuXXwwYt2j...","[TRPGPDK12903CCC651, Bring Me To Life, 0rJ8HF2...","[TRJDMHS128F92F2A0C, I'll Meet You There, 2XGM...","[TRLVQME128F931BAF3, Vanilla Twilight, 0hXBVbr...","[TRUGOGT128F92F29E9, Captains and Cruise Ships...","[TRCXWLU128F92F2A0D, This Is The Future, 17jG9...","[TRRVJCK12903CD2DCB, U Smile, 0KDJBhhe2OYnnoJt...","[TRCJAHJ128E07815B6, Stacy's Mom, 0b5Z4MPCgSFm...","[TRPWIGO128F931BAEB, Dental Care, 1IyackM7hvB1...","[TRNEITZ128F92F29EA, Designer Skyline, 30KmLL3...","[TRMIHFS128F92F2A01, Early Birdie, 1TvtrJ6uyfQ...","[TRRLGDR128F933A7C9, Injection, 0it4CBT8IGSbXv...","[TRLNFKN128F931BAF2, The Tip Of The Iceberg, 1..."
000d80cd9b58a8f77b33aa613dcfc5cbf1daf5e8,"[TRDYYKS128F4275626, Music Is Happiness, 5eWkK...","[TRBHLYP12903D0D107, 4X4, 21SudxOkg2z2LMBrghl7...","[TRABFDT12903CADD73, Up Up & Away, 0InFAWpnO2z...","[TRLNVSC12903CADD67, Simple As..., 04nE0pNbhPQ...","[TRKOCXI128F9316B54, Harmony One, 1BtLEUri7ROn...","[TRSEFCM128F429354D, Set It Up, 1sF9FiOQivhged...","[TRUWANM128F1485EE2, LDN, 016gjTKLZX8Sgaos4DRq...","[TRXKEMH128F423381D, Superfresh, 0mCoxFFYs0TRZ...","[TREMDON128F427C701, Crimewave (Crystal Castle...","[TRHPKWO128F92E01D5, The Lightning Strike, 1rE...","[TRPONOG128F4275608, The Adjustor, 6sC8fTO6Ja6...","[TRJGDTG128F421CE22, Lights & Music, 0FezhHZVm...","[TROTYPC128E07940AB, Door Peep, 0ceGoYvdbcsRll...","[TRPXIWX128F429831F, One Minute to Midnight, 0...","[TROINZB128F932F740, Crazy in Love, 0klMKiGV38...","[TROUAEG128F429354A, Message Stick, 0jFN4WAx76...","[TRQEBRP12903CADD6C, Sky Might Fall, 2Pq2jkcG8...","[TROTWMO128F42B9238, Iconography, 04gW4W5ziYM3...","[TRJYECB128F4230F29, Second Chances, 1WPZR8Kf1...","[TRJLGXB128F93043EA, Colourful, 21rILkLpA1vsYZ..."
000e9296161b73a1821aaed3d7f50d95e8665bf6,"[TROPEIV128F428F5A8, Fatal, 5HeBXKvt8Kc9wY7rrk...","[TRIAZQY128F934D58D, El Pueblo Unido, 6M3ONz42...","[TRMKAZB128F92F2F3E, Can't Keep, 08SE6CEP3gjL9...","[TRPHDFT128F92C5A75, So Com Voce, 1f0V4eqYAmy1...","[TRNXBBR128F425ECE3, We Came Along This Road, ...","[TRKPWGR128E078EE06, Where Did You Sleep Last ...","[TRLPOFY128F425ECE8, Darker With the Day, 1PKj...","[TRCHYZB128F425ECE1, The Sorrowful Wife, 3DFrC...","[TRXEAZB128E078EDCE, Something In The Way, 7hh...","[TRFVSOZ128F4281933, I'm Sleeping in a Submari...","[TRDMUWU128E078EDDB, Dumb, 13noTim30TG19L0rg9f...","[TRDRFVY128F4281937, Headlights Look Like Diam...","[TRIPLBA128F427200F, My Moon My Man, 0Bl

### Train and Test Split

In [50]:
# Train and test splits for playlist
X = arr_np[:,:-1,:]
Y = arr_np[:,1:,:]
x_train, x_val, y_train, y_val = train_test_split(X,Y,train_size=0.9,random_state=3000)
x_train, x_test, y_train, y_test = train_test_split(x_train,y_train,train_size=0.9,random_state=3000)


In [51]:
print(len(x_train[0]))

19


In [52]:
# Original Playlists
ops_x_train, ops_y_train, ops_x_val, ops_y_val, ops_x_test, ops_y_test = [], [], [], [], [], []

# This only works based on size if val and test sets switch in size switch them in these loops
for user in range(np.ma.size(x_train, axis=0)):
    names_x_train, names_y_train, names_x_val, names_y_val, names_x_test, names_y_test = [], [], [], [], [], []
    for song in range(np.ma.size(x_train, axis=1)):
        names_x_train.append(x_train[user, song, 0:3])
        names_y_train.append(y_train[user, song, 0:3])
        try:
            names_x_val.append(x_val[user, song, 0:3])
            names_y_val.append(y_val[user, song, 0:3])
        except IndexError:
            continue
        try:
            names_x_test.append(x_test[user, song, 0:3])
            names_y_test.append(y_test[user, song, 0:3])
        except IndexError:
            continue

    ops_x_train.append(names_x_train)
    ops_y_train.append(names_y_train)
    if not names_x_val:
        continue
    ops_x_val.append(names_x_val)
    ops_y_val.append(names_y_val)
    if not names_x_test:
        continue
    ops_x_test.append(names_x_test)
    ops_y_test.append(names_y_test)
x_train = x_train[:, :, 3:].astype(np.float64)
y_train = y_train[:, :, 3:].astype(np.float64)
x_val = x_val[:, :, 3:].astype(np.float64)
y_val = y_val[:, :, 3:].astype(np.float64)
x_test = x_test[:, :, 3:].astype(np.float64)
y_test = y_test[:, :, 3:].astype(np.float64)

### Define the Model

In [53]:
if os.path.exists('misc/mae_optimized_model.keras'):
    print("using saved model")
    model = load_model('misc/mae_optimized_model.keras')
else:
    print("training model")
    model = Sequential()
    model.add(Input(shape=(None,8)))
    model.add(SimpleRNN(
        16,
        activation='linear',
        return_sequences=True,
        kernel_initializer='random_uniform',
    ))
    model.add(SimpleRNN(
        16,
        activation='linear',
        return_sequences=True,
        kernel_initializer='random_uniform',
    ))
    model.add(Dense(8, activation='linear', kernel_initializer='random_uniform',))
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(torch.cuda.get_device_name(0))
    
    model.compile(loss='mae', optimizer='adam')
    model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))
    model.save('misc/mae_optimized_model.keras')

using saved model


In [54]:
mae_optimized_model_adam = model

In [55]:
def predict_sample(sample,model):
    return (model.predict(np.array([sample]))) #[0,-1]

### Run RNN

In [65]:
print('Selecting a random index in our test dataset: ')
# random_index = random.randint(0,len(x_test)-1)
random_index = random.randint(0,len(x_test)-1)
# print(x_test[random_index, :5, :])
x_test_subset = x_test[random_index, :, :]
# for test in x_test_subset:
predicted = predict_sample(x_test_subset, mae_optimized_model_adam)
print(random_index)

print('Input: ')
print(x_test[random_index])
# print(x_test[random_index, :5, :])

print('\n','Output: ')
# for i in range(0, x_test[10]):
# predicted = predict_sample(x_test, mae_optimized_model_adam)
print(predicted)

Selecting a random index in our test dataset: 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1715
Input: 
[[ 6.4100e-01  6.0700e-01 -1.0082e+01  2.5200e-02  9.1900e-02  2.0400e-05
   1.2400e-01  9.4600e-01]
 [ 5.8900e-01  5.3600e-01 -7.8700e+00  2.5000e-02  5.1800e-01  4.7700e-02
   1.1900e-01  5.7300e-01]
 [ 4.9100e-01  9.6000e-01 -5.9370e+00  4.5800e-02  2.7100e-02  3.5800e-03
   8.7000e-02  9.0900e-01]
 [ 6.2100e-01  4.2400e-01 -9.9940e+00  2.8400e-02  3.1800e-02  2.7600e-01
   9.9100e-02  5.5300e-01]
 [ 5.1700e-01  5.1100e-01 -8.1910e+00  3.3900e-02  1.9000e-01  8.2700e-03
   1.4000e-01  1.7900e-01]]

 Output: 
[ 5.3903645e-01  5.5294120e-01 -7.9529691e+00  3.7033804e-02
  1.8946734e-01  4.6870764e-03  1.2867567e-01  4.6359539e-01]


In [114]:
distance_frame = song_features_data.drop(['artist','tags','tempo','duration_mins','mode'], axis=1)
distance_frame.head()

,track_id,name,spotify_id,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence
0,TRIOREW128F424EAF0,Mr. Brightside,09ZQ5TmUG8TSL56n0knqrj,0.355,0.918,-4.360,0.0746,0.001190,0.000000,0.0971,0.240
1,TRRIVDJ128F429B0E8,Wonderwall,06UfBBDISthj1ZJAtX4xjj,0.409,0.892,-4.373,0.0336,0.000807,0.000000,0.2070,0.651
2,TROUVHL128F426C441,Come as You Are,0keNu0t0tqsWtExGM3nT1D,0.508,0.826,-5.783,0.0400,0.000175,0.000459,0.0878,0.543
3,TRUEIND128F93038C4,Take Me Out,0ancVQ9wEcHVd0RrGICTE4,0.279,0.664,-8.851,0.0371,0.000389,0.000655,0.1330,0.490
4,TRLNZBD128F935E4D8,Creep,01QoK9DA7VTeTSE3MNzp4I,0.515,0.430,-9.935,0.0369,0.010200,0.000141,0.1290,0.104


In [115]:
distance_frame.drop_duplicates(subset='track_id', keep='first', inplace=True)
distance_frame.track_id.nunique()

50683

In [116]:
def get_distances(data, p_vector):
    names = data['name']
    distance_frame = data.drop(['name', 'spotify_id'], axis=1)
    distance_dict = distance_frame.set_index(['track_id']).to_dict('index')
    for key in distance_dict:
        distance_dict[key] = list(distance_dict[key].values())
    distance_dict = distance_calc(distance_dict, p_vector, names)
    return pd.DataFrame.from_dict(distance_dict, orient='index', columns=['id', 'distance'])

def distance_calc(dict, v1, name_list):
    distances = {}
    i = 0
    name_list = name_list.to_list()
    for id in dict.keys():
        v2 = dict[id]
        value = 0.0
        for n in range(len(v1)):
            value += np.linalg.norm(v1[n] - v2[n])
        distances[name_list[i]] = (id, value)
        i += 1
    return distances

distance_frame2 = get_distances(distance_frame, predicted)


In [117]:
POTENTIAL_N = 50 #defines size

potential_songs = distance_frame2.nsmallest(POTENTIAL_N, columns='distance', keep='all')
print(potential_songs.shape)

(50, 2)


In [118]:
test_playlists = np.asarray(ops_x_test)
print(test_playlists)

[[['TRAVRKY128F429EA68' 'Loose Translation' '5Qu79Xa89JOQyZgMKWYtRk']
  ['TRBJRZH128F429EA62' 'The Electric Version' '1n0tppeA8MEOVtckhUMnmL']
  ['TRNJLKP128F427CE28' 'Aerodynamic' '168MSP7Cpz7uKGt5Qd9F16']
  ...
  ['TRHRFIN128F425DEF1' '4th Dimensional Transition'
   '14Hlfk7LY4TOw450LaVteS']
  ['TRFUIGN128F92FE2C4' 'Sea Song' '0uoM5QDvJBPu6MGrtbw0gP']
  ['TRAALAH128E078234A' 'Bitter Sweet Symphony' '0jLnevC3Vn34qVWrAa4X6x']]

 [['TRVANOX128F92FA7F4' 'Kindly Unspoken' '2OjvaNkWd0WMJSXybZcSN3']
  ['TRGCHLH12903CB7352' 'Party In The U.S.A.' '0bz2Uy1KE7bNGsGQU9pZrU']
  ['TRAGSGF128E0780CA7' 'One I Love' '5YahO7aySL2lyj6DW2tS9q']
  ...
  ['TRKJYHD128F931FB01' 'Gonna Get Caught' '3Ep2KFN6dNTJOiq2chORs2']
  ['TRARMHA128F92E7D32' "I'm Not a Loser" '1lpQGVYtbGHCGlFbwKI7yE']
  ['TRZALGW128F4296174' 'Nothing Better' '5innFMtY3a7f2PvUZsfCWQ']]

 [['TRBCALP128F933905E' 'Brooklyn Bound' '3Ui3JTqJosT2MqvC9UBLwW']
  ['TREYXVK12903CB4503' 'The Desperate Man' '22B5LxWF4DIjwhlyQq5qOl']
  ['TRIEBEZ128F9

In [119]:
print(ops_x_test[0])

[array(['TRAVRKY128F429EA68', 'Loose Translation',
       '5Qu79Xa89JOQyZgMKWYtRk'], dtype='<U109'), array(['TRBJRZH128F429EA62', 'The Electric Version',
       '1n0tppeA8MEOVtckhUMnmL'], dtype='<U109'), array(['TRNJLKP128F427CE28', 'Aerodynamic', '168MSP7Cpz7uKGt5Qd9F16'],
      dtype='<U109'), array(['TRAEUFF128F424EAAF', "I've Underestimated My Charm (Again)",
       '0gv6qCVLKKRIFaR4OC3cGh'], dtype='<U109'), array(['TRRYMIR128F9354778', 'Via con me', '053MPrQZlf9X4KYmcJ4vsd'],
      dtype='<U109'), array(['TRMGOFF128F1458362', 'Wild Frontier', '0mBdiyULKljvBLVsrS6gfp'],
      dtype='<U109'), array(['TRDJQLI128E07917C7', "Rikki Don't Lose That Number",
       '028NYvJAD47k4iGSlpT2zy'], dtype='<U109'), array(['TRYATJD128F92FDB02', 'Hazy Shade Of Winter',
       '0AJTCza5wu1jZ6aWNoZXRT'], dtype='<U109'), array(['TRXAVEU128E078D3CD', 'Oh, Goddamnit', '1KF1b9bbBYImNnC3YO2GG4'],
      dtype='<U109'), array(['TRAFUNV128F92CFEB2', 'MIA', '2hxweZgog2IrkKP86KGATI'],
      dtype='<U109'), arr

In [120]:
lyrics_embeddings_csv = pd.read_csv('misc/lyrics_embeddings.csv')
lyrics_embeddings_3d_csv = pd.read_csv('misc/lyrics_embeddings_3d.csv')

In [121]:
lyrics_embeddings = dict()
lyrics_embeddings_3d  = dict()
for idx, row in lyrics_embeddings_csv.iterrows():
    lyrics_embeddings[row[0]] = np.array(row[1:])

for idx, row in lyrics_embeddings_3d_csv.iterrows():
    lyrics_embeddings_3d[row[0]] = np.array(row[1:])


C:\Users\cam-m\AppData\Local\Temp\ipykernel_15768\1601597408.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  lyrics_embeddings[row[0]] = np.array(row[1:])
C:\Users\cam-m\AppData\Local\Temp\ipykernel_15768\1601597408.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  lyrics_embeddings_3d[row[0]] = np.array(row[1:])


In [122]:
candidates = dict()
rnn_track_ids = []
for track in ops_x_test[random_index]:
    rnn_track_ids.append(track[0])
    candidates[track[0]] = lyrics_embeddings_3d[track[0]]

cutoff = len(candidates)

for idx, row in potential_songs.head(100).iterrows():
    candidates[row['id']] = lyrics_embeddings_3d[row['id']]

len(candidates)

69

In [123]:
# For reducing dimensions of the embeddings
raw_embeddings = np.concatenate(list(lyrics_embeddings.values())).reshape(len(lyrics_embeddings), 768)
track_ids = list(lyrics_embeddings.keys())
dim_model = PCA(n_components=150, random_state=42)
dim_model.fit(raw_embeddings)
reduced_embeddings = dim_model.transform(raw_embeddings)
reduced_embeddings_dict = {track_ids[i]: reduced_embeddings[i] for i in range(len(track_ids))}

og_embeddings = np.array([reduced_embeddings_dict[track_id] for track_id in rnn_track_ids])

At this stage, we must compare the embeddings in the predicted list against those in the original input list and find the best candidates
### Cosine Similarity

In [124]:
similarities = list()

for track_id in potential_songs['id']:

    candidate_embedding = reduced_embeddings_dict[track_id].reshape(1, -1)
    similarity = cosine_similarity(candidate_embedding, og_embeddings)
    similarities.append(np.mean(similarity))

similarities = np.array(similarities)
most_similar_indices = np.argsort(similarities)[::-1]
selected_songs_cs = potential_songs.iloc[most_similar_indices[:10]]
selected_songs_cs

,id,distance
Oh Messy Life,TRAQYPC12903CDEC02,0.311424
2113,TREOZBH128F92E605A,0.377958
Phobia,TRLNNMN128F42693A3,0.359223
April Grove,TRRHBLR128F4213E11,0.284570
Follow The Light,TRVELOJ128F93256C0,0.339208
She's Nirvana,TRMFBUD128F92F6265,0.306645
Rockin' Chair,TRRMHJA128F9303E2B,0.372989
I Can't Stand,TREEBNV128F932F45A,0.326262
American Flag,TRRVUCK128F429E9B5,0.372055
Paper Cuts,TRLOEIJ128F931AD02,0.320815


### Pairwise Distances

In [125]:
candidate_embeddings = np.array([reduced_embeddings_dict[track_id] for track_id in selected_songs_cs['id']])

distances = pairwise_distances(candidate_embeddings, og_embeddings, metric='euclidean')
mean_distances = np.mean(distances, axis=1)
closest_candidates_indices = np.argsort(mean_distances)[:10]
selected_songs_pd = selected_songs_cs.iloc[closest_candidates_indices]
selected_songs_pd

,id,distance
American Flag,TRRVUCK128F429E9B5,0.372055
Paper Cuts,TRLOEIJ128F931AD02,0.320815
April Grove,TRRHBLR128F4213E11,0.284570
She's Nirvana,TRMFBUD128F92F6265,0.306645
2113,TREOZBH128F92E605A,0.377958
Phobia,TRLNNMN128F42693A3,0.359223
Follow The Light,TRVELOJ128F93256C0,0.339208
I Can't Stand,TREEBNV128F932F45A,0.326262
Rockin' Chair,TRRMHJA128F9303E2B,0.372989
Oh Messy Life,TRAQYPC12903CDEC02,0.311424


In [126]:
closest_candidates_indices

array([8, 9, 3, 5, 1, 2, 4, 7, 6, 0], dtype=int64)

In [127]:
#Predicted Data
rec_songs = song_features_data[song_features_data['track_id'].isin(selected_songs_pd['id'])]

In [128]:
rec_songs.head(10)

,track_id,name,artist,spotify_id,tags,danceability,energy,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_mins
8553,TRLOEIJ128F931AD02,Paper Cuts,Nirvana,2ilmU6mieDn7ztBKjYCxdf,"alternative_rock, grunge",0.559,0.757,-6.349,1,0.0424,0.003610,0.057200,0.1160,0.331,102.169,4.097550
12903,TRRVUCK128F429E9B5,American Flag,Cat Power,6L79kobWVIg7WxzUhkWUft,"indie, female_vocalists, folk, singer_songwriter",0.464,0.672,-6.254,1,0.0790,0.079000,0.000162,0.1030,0.395,141.518,3.513767
13693,TRAQYPC12903CDEC02,Oh Messy Life,Cap'n Jazz,3bSLvSyE9384MWc6qxk2I1,"punk, hardcore, emo",0.472,0.907,-6.386,1,0.0545,0.000691,0.000532,0.1020,0.376,107.294,2.048000
18334,TREOZBH128F92E605A,2113,Coheed and Cambria,0MFEW9QSrRgtdrqINJavJB,progressive_rock,0.396,0.798,-6.232,0,0.0586,0.006460,0.000412,0.0982,0.372,74.767,9.776000
24118,TRLNNMN128F42693A3,Phobia,Kreator,0Nqf0k55F3rTMb1q7ZXkzt,thrash_metal,0.435,0.927,-6.302,1,0.0565,0.000230,0.000004,0.1460,0.456,192.336,3.375550
27311,TRMFBUD128F92F6265,She's Nirvana,Xandria,4ptGbqozoPvXrEh9Nja1ft,"german, gothic_metal, gothic, symphonic_metal",0.491,0.712,-6.422,1,0.0310,0.000179,0.000022,0.1280,0.352,135.026,3.370000
31269,TRRHBLR128F4213E11,April Grove,Martina Topley-Bird,2doN8qhwRDKMAO0ekoAG6c,"trip_hop, lounge",0.489,0.739,-6.337,0,0.0313,0.001510,0.000044,0.0601,0.365,99.981,3.527767
32401,TRVELOJ128F93256C0,Follow The Light,Travis,34BFKNP3xcF8BiXv93Zg1q,"rock, indie, indie_pop, love, britpop",0.533,0.730,-6.407,1,0.0275,0.000137,0.000000,0.1200,0.340,111.564,3.148217
39343,TRRMHJA128F9303E2B,Rockin' Chair,Oasis,06IgBNkn56fC4ci8GP5mQX,"rock, britpop",0.522,0.856,-6.402,0,0.0334,0.000727,0.000008,0.0444,0.367,126.359,4.600000
47673,TREEBNV128F932F45A,I Can't Stand,Zola Jesus,0w2dLCq2QOzWrt35pZuYtR,NaN,0.460,0.725,-6.385,1,0.0355,0.019200,0.091300,0.0916,0.433,89.954,4.205100


In [129]:
#Original Playlist
og_songs = song_features_data[song_features_data['track_id'].isin(rnn_track_ids)]

In [130]:
og_songs.head(19)

,track_id,name,artist,spotify_id,tags,danceability,energy,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_mins
44,TRAALAH128E078234A,Bitter Sweet Symphony,The Verve,0jLnevC3Vn34qVWrAa4X6x,"rock, alternative, indie, pop, alternative_roc...",0.383,0.907,-5.409,1,0.0405,0.031300,0.000000,0.3470,0.5140,171.052,5.972217
486,TRHRFIN128F425DEF1,4th Dimensional Transition,MGMT,14Hlfk7LY4TOw450LaVteS,"rock, electronic, alternative, indie, indie_ro...",0.381,0.939,-4.684,1,0.1250,0.042000,0.918000,0.1720,0.3350,159.970,3.966883
2112,TRNJLKP128F427CE28,Aerodynamic,Daft Punk,168MSP7Cpz7uKGt5Qd9F16,"electronic, dance, house, techno, electro, french",0.617,0.883,-9.070,1,0.0503,0.307000,0.861000,0.0907,0.1260,122.870,3.458883
2186,TRISTWT128F1488FBB,High Life,Daft Punk,098ttCNmncrO4YvqWUNMvn,"electronic, dance, house, techno, electro, french",0.653,0.869,-9.393,1,0.2170,0.004350,0.413000,0.3340,0.5150,125.207,3.363333
3112,TRIPSQP12903CE450C,Lady Dada's Nightmare,MGMT,0XwfkoJnGpRw7y3s0fWxaO,"electronic, indie, experimental, indie_rock, i...",0.358,0.492,-7.844,0,0.0299,0.552000,0.899000,0.2350,0.0666,77.994,4.519100
8795,TRDJQLI128E07917C7,Rikki Don't Lose That Number,Steely Dan,028NYvJAD47k4iGSlpT2zy,"rock, jazz, classic_rock, 70s",0.649,0.478,-13.628,1,0.0381,0.510000,0.000134,0.0915,0.5710,116.179,4.543767
8899,TRRYMIR128F9354778,Via con me,Paolo Conte,053MPrQZlf9X4KYmcJ4vsd,"jazz, singer_songwriter, piano",0.587,0.706,-9.154,0,0.2040,0.777000,0.006570,0.0929,0.7330,164.438,2.776000
10090,TRCESGY128E0780E09,Breakthru,Queen,0PesritaDvkWORoHQJmP3z,"rock, classic_rock, hard_rock, 80s, british",0.569,0.649,-13.531,1,0.0509,0.063300,0.004920,0.3640,0.3340,90.084,4.137100
10835,TRQWUUG128F1485DAB,Don't Know What You Got (Till It's Gone),Cinderella,02inbOlZndnAM99BeRvxd9,"classic_rock, hard_rock, 80s, blues_rock",0.247,0.749,-5.542,1,0.0357,0.064900,0.081800,0.7520,0.2320,130.192,4.567767
14187,TRNQUIT128F428A0BD,Time Won't Let Me Go,The Bravery,12d7zOCGENeo5v2XF295sA,"rock, alternative, indie, alternative_rock, in...",0.386,0.864,-4.776,1,0.0323,0.001330,0.000054,0.4400,0.4180,154.774,4.188667


In [132]:
rec_songs.head(5)

,track_id,name,artist,spotify_id,tags,danceability,energy,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_mins
8553,TRLOEIJ128F931AD02,Paper Cuts,Nirvana,2ilmU6mieDn7ztBKjYCxdf,"alternative_rock, grunge",0.559,0.757,-6.349,1,0.0424,0.003610,0.057200,0.1160,0.331,102.169,4.097550
12903,TRRVUCK128F429E9B5,American Flag,Cat Power,6L79kobWVIg7WxzUhkWUft,"indie, female_vocalists, folk, singer_songwriter",0.464,0.672,-6.254,1,0.0790,0.079000,0.000162,0.1030,0.395,141.518,3.513767
13693,TRAQYPC12903CDEC02,Oh Messy Life,Cap'n Jazz,3bSLvSyE9384MWc6qxk2I1,"punk, hardcore, emo",0.472,0.907,-6.386,1,0.0545,0.000691,0.000532,0.1020,0.376,107.294,2.048000
18334,TREOZBH128F92E605A,2113,Coheed and Cambria,0MFEW9QSrRgtdrqINJavJB,progressive_rock,0.396,0.798,-6.232,0,0.0586,0.006460,0.000412,0.0982,0.372,74.767,9.776000
24118,TRLNNMN128F42693A3,Phobia,Kreator,0Nqf0k55F3rTMb1q7ZXkzt,thrash_metal,0.435,0.927,-6.302,1,0.0565,0.000230,0.000004,0.1460,0.456,192.336,3.375550


In [134]:
rec_songs = rec_songs.loc[:, ['name', 'danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness',
                                'liveness', 'valence']]
rec_songs.head(10)

,name,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence
8553,Paper Cuts,0.559,0.757,-6.349,0.0424,0.003610,0.057200,0.1160,0.331
12903,American Flag,0.464,0.672,-6.254,0.0790,0.079000,0.000162,0.1030,0.395
13693,Oh Messy Life,0.472,0.907,-6.386,0.0545,0.000691,0.000532,0.1020,0.376
18334,2113,0.396,0.798,-6.232,0.0586,0.006460,0.000412,0.0982,0.372
24118,Phobia,0.435,0.927,-6.302,0.0565,0.000230,0.000004,0.1460,0.456
27311,She's Nirvana,0.491,0.712,-6.422,0.0310,0.000179,0.000022,0.1280,0.352
31269,April Grove,0.489,0.739,-6.337,0.0313,0.001510,0.000044,0.0601,0.365
32401,Follow The Light,0.533,0.730,-6.407,0.0275,0.000137,0.000000,0.1200,0.340
39343,Rockin' Chair,0.522,0.856,-6.402,0.0334,0.000727,0.000008,0.0444,0.367
47673,I Can't Stand,0.460,0.725,-6.385,0.0355,0.019200,0.091300,0.0916,0.433


In [146]:
fig = go.Figure()

text_data = list(candidates.keys())
embeddings_3d = np.concatenate(list(candidates.values())).reshape(len(candidates), 3)

color_data = ['blue' if i < cutoff else 'red' for i in range(len(candidates))]
for i in closest_candidates_indices:
    color_data[i] = 'green'
color_data[closest_candidates_indices[0]] = 'purple'

fig.add_trace(go.Scatter3d(
    x=embeddings_3d[:, 0],
    y=embeddings_3d[:, 1],
    z=embeddings_3d[:, 2],
    text=text_data,
    mode='markers',
    marker=dict(
        size=5,
        color=color_data,
        colorscale='Viridis',
        opacity=1
    )
))


fig.update_layout(
    scene=dict(
        xaxis=dict(title='x'),
        yaxis=dict(title='y'),
        zaxis=dict(title='z')
    ),
	width=1000,
    height=800
)
fig.update_layout(legend_title_text = "Songs")

fig.show()


### Spotify API Evaluation
Get recommendations from Spotify by passing in minimum and maximum feature values, and compare the songs to our recommended playlist.


In [138]:
from spotify import SpotifyAPI
from dotenv import load_dotenv
# Get the input playlist
# Traverse along each song in playlist and preserve min max of feature values and 
# Store in list that gets passed to parameters for Spotify APIcommendations

input_playlist = og_songs
features = dict()
feature_names = ["danceability","energy","loudness","speechiness","acousticness","instrumentalness","liveness", "valence"]

for feature in feature_names:
    features["min_" + feature] = input_playlist[feature].min()
    features["max_" + feature] = input_playlist[feature].max()

features["seed_genres"] = input_playlist['tags'].iloc[0].split(',')[:3]
features["seed_genres"] = ','.join(features["seed_genres"])
features


{'min_danceability': 0.247,
 'max_danceability': 0.718,
 'min_energy': 0.478,
 'max_energy': 0.985,
 'min_loudness': -13.628,
 'max_loudness': -3.094,
 'min_speechiness': 0.0299,
 'max_speechiness': 0.217,
 'min_acousticness': 0.000421,
 'max_acousticness': 0.777,
 'min_instrumentalness': 0.0,
 'max_instrumentalness': 0.918,
 'min_liveness': 0.0907,
 'max_liveness': 0.829,
 'min_valence': 0.0666,
 'max_valence': 0.923,
 'seed_genres': 'rock, alternative, indie'}

In [139]:
env_path = os.path.join('misc', '.env')
load_dotenv(dotenv_path=env_path)
SPOTIFY_CREDS = [os.getenv('SPOTIFY_CLIENT_ID'), 
                os.getenv('SPOTIFY_CLIENT_SECRET')]

if not all(SPOTIFY_CREDS):
        print("Please set the SPOTIFY_CLIENT_ID and SPOTIFY_CLIENT_SECRET environment variables")
        exit(1)

spotify = SpotifyAPI(*SPOTIFY_CREDS)
SPOTIFY_CREDS

['1fa1ca9358074af48f6230e7b13c0737', '880a92c5194f496b929d0d4d96ec3a59']

In [140]:
recs_from_spotify = spotify.get_recommendations(features=features, limit=10)

In [141]:
# SPOTIFY's PREDICTED RECOMMENDATIONS
recs_from_spotify = recs_from_spotify.loc[:, ['name', 'danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness',
                                'liveness', 'valence']]
recs_from_spotify

,name,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence
0,Aces High - 1998 Remastered Version,0.305,0.974,-5.079,0.0639,0.000937,0.013900,0.153,0.538
1,I Will Wait,0.481,0.766,-5.208,0.0334,0.010300,0.006700,0.302,0.436
2,Africa,0.500,0.770,-5.059,0.0432,0.238000,0.006360,0.104,0.633
3,Whatever It Takes,0.686,0.862,-3.426,0.0340,0.002070,0.000011,0.148,0.472
4,Numb / Encore,0.687,0.793,-4.254,0.1660,0.060300,0.000000,0.582,0.751
5,Bitter Sweet Symphony - Remastered 2016,0.356,0.917,-6.006,0.0479,0.033500,0.000000,0.406,0.518
6,Break Stuff,0.432,0.884,-6.657,0.1280,0.012800,0.000457,0.166,0.619
7,Hold the Line,0.473,0.898,-4.797,0.0561,0.007830,0.061800,0.173,0.808
8,Numb,0.488,0.870,-4.185,0.0421,0.003530,0.000000,0.484,0.211
9,Someday,0.542,0.531,-7.481,0.0301,0.003300,0.000002,0.338,0.671


In [142]:
# OUR PREDICTED RECOMMENDATIONS
rec_songs

,name,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence
8553,Paper Cuts,0.559,0.757,-6.349,0.0424,0.003610,0.057200,0.1160,0.331
12903,American Flag,0.464,0.672,-6.254,0.0790,0.079000,0.000162,0.1030,0.395
13693,Oh Messy Life,0.472,0.907,-6.386,0.0545,0.000691,0.000532,0.1020,0.376
18334,2113,0.396,0.798,-6.232,0.0586,0.006460,0.000412,0.0982,0.372
24118,Phobia,0.435,0.927,-6.302,0.0565,0.000230,0.000004,0.1460,0.456
27311,She's Nirvana,0.491,0.712,-6.422,0.0310,0.000179,0.000022,0.1280,0.352
31269,April Grove,0.489,0.739,-6.337,0.0313,0.001510,0.000044,0.0601,0.365
32401,Follow The Light,0.533,0.730,-6.407,0.0275,0.000137,0.000000,0.1200,0.340
39343,Rockin' Chair,0.522,0.856,-6.402,0.0334,0.000727,0.000008,0.0444,0.367
47673,I Can't Stand,0.460,0.725,-6.385,0.0355,0.019200,0.091300,0.0916,0.433


In [143]:
def calculate_euclidean_distance(v1, v2):
    return np.linalg.norm(v1 - v2)

def calculate_score(pd1, pd2):
    if pd1.shape[1] != pd2.shape[1]:
        raise ValueError("Dataframes must have the same number of features.")
    
    for _, x in pd1.iterrows():
        fx = np.array(x[1:].values)
        dist = list()
        for _, y in pd2.iterrows():
            fy = np.array(y[1:].values)
            dist.append(calculate_euclidean_distance(fx, fy))
        print(np.mean(dist))


In [144]:
calculate_score(rec_songs, recs_from_spotify)

1.5059530332464948
1.4456050606223698
1.5129869289765172
1.4261873024527538
1.4463676036912252
1.542412471309567
1.4978183257167976
1.5373330718185432
1.5354420740893615
1.5126993462504639
